## 带标记的目标实时跟踪实践  
原理：提取视频中的一帧图片，然后对其进行图像二值化,然后找图中所有的黑色连通域和白色连通域，最终计算这些黑色连通域和白色连通域的中心点。由于是黑白相间的环，所以那些两种中心点非常接近的位置则是圆环所在位置。最终实现了对视频中的白色环进行了实时跟踪。

In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
import time

##### 读取视频流

In [26]:
videoCap = cv2.VideoCapture('./oneCCC.wmv')
if videoCap.isOpened() == False:
    print("Error opening video stream or file")

while videoCap.isOpened():
    ret, frame = videoCap.read()
    if ret == False:
        print('video end')
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 转为灰度图
    frame = cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1] # 二值化
    ret, labels = cv2.connectedComponents(frame) # 求连通域

    center1 = np.zeros((ret, 2)).astype(np.int32) # 黑色连通域中心点

    # 估计各个黑色连通域的中心点
    for i in range(ret):
        row, col = np.where(labels == i)
        center1[i] = np.mean(row), np.mean(col)
    frame = cv2.bitwise_not(frame) # 反色
    ret, labels = cv2.connectedComponents(frame) # 求连通域

    center2 = np.zeros((ret, 2)).astype(np.int32) # 白色连通域中心点
    for i in range(ret):
        row, col = np.where(labels == i)
        center2[i] = np.mean(row), np.mean(col)


    index = 0
    value = np.inf
    # 找到黑色连通域和白色连通域中心点距离最近的两个点
    for i in range(ret):
        # center1(ret+1, 2), center2[i](1, 2)
        mvalue = np.sum(
            np.abs(center1 - center2[i]), axis=1
        ).min() # sum是计算曼哈顿距离
        if mvalue < value:
            value = mvalue
            index = i
    # 绘制出目标框
    y, x = np.where(labels == index)
    cv2.rectangle(
        frame, 
        (np.min(x), np.min(y)), 
        (np.max(x), np.max(y)), 
        165,   # 灰度图像颜色格式
        3
    )

    # plt单元格展示
    plt.imshow(frame)
    plt.show()
    time.sleep(0.05)
    clear_output(wait=True) # 清除输出，避免输出过多

    ## cv2窗口展示
    # cv2.imshow('track circle', frame)
    # cv2.waitKey(0)
    # if cv2.waitKey(25) & 0xFF == ord('q'): #按下q键就退出
    #     break

videoCap.release() # 释放视频捕获对象所占用的资源
cv2.destroyAllWindows() # 关闭所有OpenCV窗口

KeyboardInterrupt: 